In [ ]:
from google.colab import drive

drive.mount('/content/drive')
# Đọc dữ liệu file_path = 'cleaned_data.csv'
directory_path = '/content/drive/My Drive/[BI Season 8] Workspace/ROUND 2/datasets/'


Mounted at /content/drive


## Data preprocessing


In [ ]:
import pandas as pd
import os
import numpy as np
import re


def join_csv_OM():
    dir_path = 'OM\OM'
    files = os.listdir(dir_path)
    csv_file = files.pop(0)
    df = pd.read_csv(dir_path + '\\'+ csv_file, encoding='utf-8')
    dtype_spec = (df.dtypes).to_dict()
    dtype_spec['from_city'] = 'string'
    dtype_spec['to_city'] = 'string'
    for k in dtype_spec.keys():
        if dtype_spec[k] == 'int64':
            dtype_spec[k] = 'float64'
    # print(dtype_spec)
    df = pd.read_csv(dir_path + '\\'+ csv_file, encoding='utf-8', dtype=dtype_spec)
    for file in files:
        df2 = pd.read_csv(dir_path + '\\'+ file, encoding='utf-8', dtype=dtype_spec)
        df = pd.concat([df, df2], ignore_index=True)
        print(file)
    print(df.tail())
    df = df.drop_duplicates()
    df.to_csv('OM.csv', index=False, encoding='utf-8')

def join_csv_LM():
    dir_path = 'LM\LM'
    files = os.listdir(dir_path)
    csv_file = files.pop(0)
    df = pd.read_csv(dir_path + '\\'+ csv_file, encoding='utf-8')
    # print(dtype_spec)
    df = pd.read_csv(dir_path + '\\'+ csv_file, encoding='utf-8')
    for file in files:
        df2 = pd.read_csv(dir_path + '\\'+ file, encoding='utf-8')
        df = pd.concat([df, df2], ignore_index=True)
        print(file)
    print(df.tail())
    df = df.drop_duplicates()
    df['items'] = df['items'].apply(lambda x: remove_next_line(x))
    print(df['items'][12948])
    # Xóa 1 dòng order_id 238265391
    df.to_csv('OM_cleaned.csv', index=False, encoding='utf-8')


def remove_next_line(s):
    if pd.notna(s):
        s = re.sub(r"\r", " ", s)
        return re.sub(r"\n", " ", s)

csv_file_path = 'order_item.csv'
df = pd.read_csv(csv_file_path, encoding = 'utf-8')
df = df.drop(columns=['Unnamed: 0'])
grouped_df = df.groupby(['order_id', 'item_name'], as_index=False).agg({'item_quantity': 'sum'})
print(grouped_df)


## Tách Order Items

In [ ]:
import re
import json
import pandas as pd
import ast
import numpy as np
from tqdm import tqdm


def is_nan(value):
    if isinstance(value, float):  # Check if the value is a float
        return np.isnan(value)  # Use np.isnan only for float types
    elif value is None:  # Check for None
        return True
    else:
        return False

csv_file_path = 'OM_cleaned.csv'
df = pd.read_csv(csv_file_path, encoding = 'utf-8')

def extract_pattern(text, pattern):
    if text is None:
        return None
    pattern = re.compile(pattern)
    match = pattern.search(text)
    if match is None:
        return None
    else:
        return match.group(0)

order_id = []
item_id = []
item_name = []
item_quantity = []
item_price = []
is_dangerous = []
k = []
price_pattern = r'[0-9]+(\.[0-9]+)?VND'
quantity_pattern = r'\) x[0-9]+ \('
item_id_pattern = r'[0-9A-Z]+_?VNAMZ-[0-9]+'
name_excluded_pattern = r'\([0-9A-Z]+_?VNAMZ-[0-9]+\) x[0-9]+ \([0-9]+(\.[0-9]+)?VND\)?'
for  indexa, row in tqdm(df.iterrows()):
    # print(row)
    item_order_id = row['order_id']
    item_line = row['items']
    if is_nan(item_line): continue
    elif re.match(r'\[{"item_name"', str(item_line)):
        try:
            items_list = ast.literal_eval(str(item_line))
            for i in items_list:
                order_id.append(item_order_id)
                item_id.append(None)
                item_price.append(None)
                is_dangerous.append(None)
                if 'item_name' in i.keys():
                    item_name.append((i['item_name']).strip())
                else:
                    item_name.append(None)
                if 'item_quantity' in i.keys():
                    item_quantity.append(int(i['item_quantity']))
                else:
                    item_quantity.append(None)
        except Exception as e:
            print(f"Error: {e}")
            print(f'Order_id: {item_order_id} Line: {item_line}')
    elif re.match(r'\[{"item_description"', str(item_line)):
        try:
            corrected_item_line = re.sub(r'\bfalse\b', 'False', str(item_line))
            corrected_item_line = re.sub(r'\btrue\b', 'True', corrected_item_line)
            corrected_item_line = re.sub(r',null', '', corrected_item_line)
            items_list = ast.literal_eval(corrected_item_line)
            for i in items_list:
                order_id.append(item_order_id)
                item_id.append(None)
                item_price.append(None)
                if 'item_description' in i.keys():
                    item_name.append(i['item_description'].strip())
                else:
                    item_name.append(None)
                if 'quantity' in i.keys():
                    item_quantity.append(int(i['quantity']))
                else:
                    item_quantity.append(None)
                if 'is_dangerous_good' in i.keys():
                    is_dangerous.append(i['is_dangerous_good'])
                else:
                    is_dangerous.append(None)
        except Exception as e:
            print(f"Error: {e}")
            print(f'Order_id: {item_order_id} Line: {item_line}')
    else:
        items_list = str(item_line).split('), ')
        for i in items_list:
            price = extract_pattern(extract_pattern(i, price_pattern), r'[0-9]+')
            quantity = extract_pattern(extract_pattern(i, quantity_pattern), r'[0-9]+')
            id = extract_pattern(i, item_id_pattern)
            name = (re.sub(name_excluded_pattern, "", i)).strip()
            if price is None:
                item_price.append(None)
            else:
                item_price.append(int(price))
            if quantity is None:
                item_quantity.append(None)
            else:
                item_quantity.append(int(quantity))
            if id is None:
                item_id.append(None)
            else:
                item_id.append(id)
            if name is None:
                item_name.append(None)
            else:
                item_name.append(name.strip())
            order_id.append(item_order_id)
            is_dangerous.append(None)

data = {'order_id': order_id, 'item_name': item_name, 'item_id': item_id, 'item_quantity': item_quantity, 'item_price': item_price, 'is_dangerous_good': is_dangerous}
df = pd.DataFrame(data)
df.to_csv('order_item.csv', encoding='utf-8')


## Order Status Analysis

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

status_col = ['created', 'pickup', 'deliver', 'rts', 'complete']

def check1(d_start, d_end, rts):
    if pd.isna(rts):
        return False
    else:
        return (rts <= d_end) & (d_start <= rts)

def get_counts(order_df, d_start):
  d_end = d_start + pd.DateOffset(days=1) - pd.DateOffset(seconds=1)
  for c in status_col:
    order_df[c] = 0
  order_df.loc[((d_start <= order_df['creation_datetime']) & (order_df['creation_datetime'] <= d_end)) | ((d_start > order_df['creation_datetime']) & (d_end < order_df['pickup_datetime'])), 'created'] = 1
  order_df.loc[((d_start <= order_df['pickup_datetime']) & (order_df['pickup_datetime'] <= d_end)) | ((d_start > order_df['pickup_datetime']) & (d_end < order_df['first_valid_delivery_attempt_datetime'])), 'pickup'] = 1
  order_df.loc[((d_start <= order_df['first_valid_delivery_attempt_datetime']) & (order_df['first_valid_delivery_attempt_datetime'] <= d_end)) | ((d_start > order_df['first_valid_delivery_attempt_datetime']) & (d_end < order_df['rts_trigger_datetime'])), 'deliver'] = 1
  order_df.loc[((d_start <= order_df['rts_trigger_datetime']) & (order_df['rts_trigger_datetime'] <= d_end)) | ((d_start > order_df['rts_trigger_datetime']) & (d_end < order_df['delivery_success_datetime'])), 'rts'] = 1
  order_df.loc[((d_start <= order_df['delivery_success_datetime']) & (order_df['delivery_success_datetime'] <= d_end)), 'complete'] = 1
  create_count = (order_df['created'] == 1).sum()
  pickup_count = (order_df['pickup'] == 1).sum()
  deliver_count = (order_df['deliver'] == 1).sum()
  rts_count = (order_df['rts'] == 1).sum()
  complete_count = (order_df['complete'] == 1).sum()
  order_df = order_df.drop(order_df[order_df['complete'] == 1].index)
  return order_df, create_count, pickup_count, deliver_count, rts_count, complete_count

def get_counts2(order_df, d_start):
  d_end = d_start + pd.DateOffset(days=1) - pd.DateOffset(seconds=1)
  for c in status_col:
    order_df[c] = 0
  order_df.loc[((d_start <= order_df['creation_datetime']) & (order_df['creation_datetime'] <= d_end)) | ((d_start > order_df['creation_datetime']) & (d_end < order_df['pickup_datetime'])), 'created'] = 1
  order_df.loc[((d_start <= order_df['pickup_datetime']) & (order_df['pickup_datetime'] <= d_end)) | ((d_start > order_df['pickup_datetime']) & (d_end < order_df['first_valid_delivery_attempt_datetime'])), 'pickup'] = 1
  order_df.loc[((d_start <= order_df['first_valid_delivery_attempt_datetime']) & (order_df['first_valid_delivery_attempt_datetime'] <= d_end)) | ((d_start > order_df['first_valid_delivery_attempt_datetime']) & (d_end < order_df['rts_trigger_datetime'])), 'deliver'] = 1
  order_df.loc[((d_start <= order_df['rts_trigger_datetime']) & (order_df['rts_trigger_datetime'] <= d_end)) | ((d_start > order_df['rts_trigger_datetime']) & (d_end < order_df['delivery_success_datetime'])), 'rts'] = 1
  order_df.loc[((d_start <= order_df['delivery_success_datetime']) & (order_df['delivery_success_datetime'] <= d_end)), 'complete'] = 1
  create_count = (order_df['created'] == 1).sum()
  pickup_count = (order_df['pickup'] == 1).sum()
  deliver_count = (order_df['deliver'] == 1).sum()
  rts_count = (order_df['rts'] == 1).sum()
  complete_count = (order_df['complete'] == 1).sum()
  order_df = order_df.drop(order_df[order_df['complete'] == 1].index)
  return order_df, create_count, pickup_count, deliver_count, rts_count, complete_count
# Load and prepare data
cal_df = pd.read_csv(directory_path+'ryo_calendar (1).csv', encoding='utf-8')
cal_df['date'] = pd.to_datetime(cal_df['date'])
order_df = pd.read_csv(directory_path+'OM_cleaned.csv', encoding='utf-8')

date_col = ['creation_datetime', 'pickup_datetime', 'first_valid_delivery_attempt_datetime', 'rts_trigger_datetime', 'delivery_success_datetime']
for c in date_col:
    order_df[c] = pd.to_datetime(order_df[c], format='mixed')
for c in status_col:
    order_df[c] = None
# Lists to store results
created = []
picked_up = []
delivered = []
rts_triggered = []
completed = []

# Calculate counts
for i, row in tqdm(cal_df.iterrows()):
    d_start = row['date']
    order_df, created_count, pickup_count , deliver_count, rts_triggered_count, completed_count = get_counts(order_df, d_start)
    print(f'{d_start}: {created_count}, {pickup_count}, {deliver_count}, {rts_triggered_count}, {completed_count}')
    created.append(created_count)
    picked_up.append(pickup_count)
    delivered.append(deliver_count)
    rts_triggered.append(rts_triggered_count)
    completed.append(completed_count)

# Add results to the calendar DataFrame
cal_df['created'] = created
cal_df['picked_up'] = picked_up
cal_df['delivered'] = delivered
cal_df['rts_triggered'] = rts_triggered
cal_df['completed'] = completed

# Print the final DataFrame
print(cal_df)
cal_df.to_csv('new_cal.csv', index=False)


NameError: name 'directory_path' is not defined